# <span style='color:red'>Project 2.  Due October 23</span>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

C:\Users\00\AppData\Local\Temp\ipykernel_32796\1189900070.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### In this project we develop a first-order algorithm to construct a portfolio using intraday data.

In [2]:
import csv
import sys
import scipy.io
import numpy as np
import math
import matplotlib.pyplot as plt

##### We will have data involving $n$ assets, and use the first $T$ days of the data to compute the portfolio.
##### The computation will produce a weight $x_i$ for each asset $i = 1,...,n$, which could be long or short.
##### We assume that on each day, a position is taken at the open, and closed at noon.  So we define:
$$ p^o_{j,t} = \ \text{price of asset $j$ on day $t$ at the open}$$
$$ p^1_{j,t} = \ \text{price of asset $j$ on day $t$ at noon}$$
$$ r_{j,t} =  \ \frac{p^1_{j,t} - p^o_{j,t}}{p^o_{j,t}} = \ \text{return earned by asset $j$ on day $t$.}$$
$$ \bar r_j = \ \frac{1}{T} \sum_{t = 1}^T r_{j,t} = \ \text{average return earned by asset $j$.}$$

#### The optimization problem to solve depends on two parameters: $\theta \ge 0$ and $\pi > 0 0$.
####
$$ \text{minimize} \ \left(-\sum_{j = 1}^n \bar r_j x_j\right) \ + \ \theta \left( \frac{1}{T} \sum_{t = 1}^{T}\left[\sum_{j = 1}^n (r_{j,t} -  \bar r_j)x_j\right]^\pi\right)^{1/\pi}$$
#### 
#### There are no constraints on the quantities $x_j$.
#### The first sum is minus the average return earned by the portfolio.  In the second sum, the quantity inside the square brackets is the excess return earned by the portfolio on day $t$, magnified by the power $\pi$.  The quantity $\theta$ is a risk aversion parameter.


### <span style='color:red'> Task 1. Develop a first-order method to address this computational problem.</span>
#### 
#### Your method should work with values of $T$ at least $100$. Use the data we provide for AMZN, NFLX, TSLA, i.e., $n = 3$. 
###
#### Make sure your code works with $\pi = 0.5, 2, 4, 6$, and $\theta = 0.1, 10, 1000, 10^5, 10^6$.

#### Data Preprocessing:

In [3]:
import pandas as pd
# construct interval data
from datetime import datetime, time
from datetime import date
import warnings

# Suppress the warning
warnings.filterwarnings("ignore")

df_TSLA = pd.read_csv('TSLA.csv', skiprows = 3)
df_TSLA = df_TSLA.loc[:, ['Dates', 'Close']].dropna()
df_TSLA['Dates'] = pd.to_datetime(df_TSLA['Dates'])
start = datetime.strptime('09:30:00', '%H:%M:%S').time()
noon = datetime.strptime('12:00:00', '%H:%M:%S').time()
df_TSLA['Date']=df_TSLA['Dates'].dt.date
df_TSLA = df_TSLA[(df_TSLA['Dates'].dt.time==noon) | (df_TSLA['Dates'].dt.time==start)]



df_AMZN = pd.read_csv('AMZN.csv', skiprows = 3)
#missing value backfill
df_AMZN['Dates'][0]='1/4/21 9:30'
df_AMZN = df_AMZN.loc[:, ['Dates', 'Close']].dropna()
df_AMZN['Dates'] = pd.to_datetime(df_AMZN['Dates'])
start = datetime.strptime('09:30:00', '%H:%M:%S').time()
noon = datetime.strptime('12:00:00', '%H:%M:%S').time()
noon_correction = datetime.strptime('12:01:00', '%H:%M:%S').time()
df_AMZN['Date']=df_AMZN['Dates'].dt.date
df_AMZN = df_AMZN[(df_AMZN['Dates'].dt.time==noon) | \
                  (df_AMZN['Dates'].dt.time==start) | \
                    ((df_AMZN['Dates'].dt.time==noon_correction) & (df_AMZN['Date'] == date(2021, 4, 20)) )\
                        |((df_AMZN['Dates'].dt.time==noon_correction) & (df_AMZN['Date'] == date(2021, 6, 14)) )]

df_NFLX = pd.read_csv('NFLX.csv', skiprows = 3)
#missing value backfill
df_NFLX['Dates'][0]='2/1/21 9:30'
df_NFLX = df_NFLX.loc[:, ['Dates', 'Close']].dropna()
df_NFLX['Dates'] = pd.to_datetime(df_NFLX['Dates'])
start = datetime.strptime('09:30:00', '%H:%M:%S').time()
noon = datetime.strptime('12:00:00', '%H:%M:%S').time()
df_NFLX['Date']=df_NFLX['Dates'].dt.date
df_NFLX = df_NFLX[(df_NFLX['Dates'].dt.time==noon) | (df_NFLX['Dates'].dt.time==start)]


mask1 = df_TSLA['Date'].isin(df_AMZN['Date'])
mask2 = df_TSLA['Date'].isin(df_NFLX['Date'])
intersection_dates = df_TSLA['Date'][mask1&mask2]

AMZN = np.array(df_AMZN[df_AMZN['Date'].isin(intersection_dates)].groupby('Date')['Close'].pct_change().dropna())[:100]
NFLX = np.array(df_NFLX[df_NFLX['Date'].isin(intersection_dates)].groupby('Date')['Close'].pct_change().dropna())[:100]
TSLA = np.array(df_TSLA[df_TSLA['Date'].isin(intersection_dates)].groupby('Date')['Close'].pct_change().dropna())[:100]


In [4]:
# Define your data, parameters, and objective function here
#r_jt = np.random.rand(T, n)  # Use random returns to test your function so that you don't have to read data
r_jt = np.vstack((AMZN,NFLX,TSLA)).T
bar_r_j = np.mean(r_jt, axis=0)  # Average returns for each asset

#### Helper Functions to Compute Objective Function and Approximate the Gradient:

In [5]:
def compute_objective(x_j, r_jt, bar_r_j, pi, theta):
    return -np.sum(bar_r_j * x_j) + theta * (np.mean((abs(np.dot(r_jt - bar_r_j, x_j)) ** pi)) ** (1/pi))

In [6]:
def approximate_gradient(x_j, r_jt, bar_r_j, pi, theta):

    change = [1e-5,0,0]
    grad1=(compute_objective(x_j+change, r_jt, bar_r_j, pi, theta) \
        - compute_objective(x_j-change, r_jt, bar_r_j, pi, theta))/(2*change[0])
    

    change = [0,1e-5,0]    
    grad2=(compute_objective(x_j+change, r_jt, bar_r_j, pi, theta) \
        - compute_objective(x_j-change, r_jt, bar_r_j, pi, theta))/(2*change[1])


    change = [0,0,1e-5]
    grad3=(compute_objective(x_j+change, r_jt, bar_r_j, pi, theta) \
        - compute_objective(x_j-change, r_jt, bar_r_j, pi, theta))/(2*change[2])

    return np.array([grad1,grad2,grad3])

In [7]:
# np.random.seed(10)
# # Initial random portfolio weights.
# n=3
# T=100
# x_j = np.random.rand(n) 
# r_jt = np.random.rand(T, n)
# bar_r_j = np.mean(r_jt, axis=0)  
# pi=0.5
# theta=0.1
# compute_objective(x_j, r_jt, bar_r_j, pi, theta)

#### First Order Method:

In [8]:

def first_order(x_j, r_jt, bar_r_j, pi, theta):
    print(' ')
    print('theta:', theta,' ', 'pi:', pi)
    # Gradient Descent terminal conditions
    max_iterations = 1000
    tolerance = 1e-4

    # Gradient Descent Hyperparameters: Backtracking Line Search with Momentum
    alpha = 0.3  # Backtracking parameter (you can adjust this)
    beta = 0.90  # Backtracking parameter (you can adjust this)
    old_delta = 0 
    mu=0.1 # Monemtum parameter (you can adjust this)

    for iteration in range(max_iterations):

        gradient = approximate_gradient(x_j, r_jt, bar_r_j, pi, theta)
        
        # Backtracking Line Search
        t = 1.0
        while compute_objective(x_j - t * gradient, r_jt, bar_r_j, pi, theta) > \
            compute_objective(x_j, r_jt, bar_r_j, pi, theta) - alpha * t * np.linalg.norm(gradient) ** 2:
            t *= beta
            if t < 1e-3 and compute_objective(x_j - t * gradient, r_jt, bar_r_j, pi, theta)<\
                compute_objective(x_j, r_jt, bar_r_j, pi, theta):
                break

        # Update portfolio weights with momentum
        gradstep = -t * gradient
        delta = gradstep + (1 - mu)*old_delta
        x_j += delta
        old_delta = old_delta

        # Check for convergence
        if abs(np.linalg.norm(gradient)) < tolerance:
            print('Converged with in', iteration+1, 'iterations')    
            break

    if abs(np.linalg.norm(gradient)) > tolerance: 
        print('Max iteration reached')
        
    print('Gradient norm:', np.linalg.norm(gradient))
    print('Objective value: ', compute_objective(x_j, r_jt, bar_r_j, pi, theta))
    print('Final_portfolio_weights ', x_j)     

    # Final portfolio allocation
    return x_j


In [9]:
pis = np.array([0.5, 2, 4, 6])
thetas = np.array([0.1, 10, 1000, 1e5, 1e6])
weights_theta_pi = np.zeros([len(thetas),len(pis),3])   

for i in range(len(thetas)):
    for j in range(len(pis)):
        n = 3  # Number of assets
        T = 100  # Number of days
        np.random.seed(10)
        # Initial random portfolio weights.
        theta = thetas[i]
        pi = pis[j]
        x_j = np.random.rand(n) 
        weights_theta_pi[i,j,:]=first_order(x_j, r_jt, bar_r_j, pi, theta)

 
theta: 0.1   pi: 0.5
Max iteration reached
Gradient norm: 0.002320547429989169
Objective value:  6.754235046413793e-06
Final_portfolio_weights  [ 0.60393063 -0.13605496 -0.15981191]
 
theta: 0.1   pi: 2.0
Max iteration reached
Gradient norm: 0.00011676914032580526
Objective value:  2.7662727841066084e-05
Final_portfolio_weights  [ 0.41653588 -0.07294781 -0.32042219]
 
theta: 0.1   pi: 4.0
Max iteration reached
Gradient norm: 0.0004029904106313674
Objective value:  9.671399108399605e-05
Final_portfolio_weights  [ 0.21279053 -0.00420672 -0.1123706 ]
 
theta: 0.1   pi: 6.0
Max iteration reached
Gradient norm: 0.0005835701876840125
Objective value:  4.5180554986642395e-05
Final_portfolio_weights  [ 0.06949977  0.00307511 -0.03397779]
 
theta: 10.0   pi: 0.5
Max iteration reached
Gradient norm: 0.0006278658224675351
Objective value:  2.4857889898773006e-08
Final_portfolio_weights  [ 4.10509670e-07 -6.94061338e-08 -1.75966317e-07]
 
theta: 10.0   pi: 2.0
Converged with in 125 iterations
Gr

### <span style='color:red'>Task 2: Benchmark your portfolio on the remaining days</span>
#### On each of the remaining days, we proceed as follows.  Denote by $x^*$ your portfolio. At the market open we invest $10^9 x^*_j$ on each asset $j$, and we close the position (by) noon.  You need to use the asset's price to compute the number of shares that you invest in, whether long or short. So the total you invest equals $$ \sum_{j = 1}^n 10^9 |x^*_j|.$$
#### Report the average return earned by your portfolio.

In [12]:
def calculate_avg_return(x):   
    # Extract percentage changes and drop NaN values
    AMZN_test = np.array(df_AMZN[df_AMZN['Date'].isin(intersection_dates)].groupby('Date')['Close'].pct_change().dropna())[100:] 
    NFLX_test = np.array(df_NFLX[df_NFLX['Date'].isin(intersection_dates)].groupby('Date')['Close'].pct_change().dropna())[100:] 
    TSLA_test = np.array(df_TSLA[df_TSLA['Date'].isin(intersection_dates)].groupby('Date')['Close'].pct_change().dropna())[100:] 
    
    # Stack the percentage changes
    return_arr = np.column_stack((AMZN_test, NFLX_test, TSLA_test))
    
    total_return = 0
    num_days = len(AMZN_test)
    
    # Normalize weights to sum to 1
    x = x / np.sum(x)
    
    for i in range(num_days):
        daily_return = 0

        for j in range(3):
            # Calculate the contribution of asset j to daily return
            daily_return += np.sum(1e9 * x[j]) * return_arr[i][j]
        
        total_return += daily_return
        
    average_return = total_return / num_days
    
    return average_return


In [13]:
for i in range(weights_theta_pi.shape[0]):
    for j in range(weights_theta_pi.shape[1]):
            print('theta:', thetas[i],' ', 'pi:', pis[j])
            print('average return: ', calculate_avg_return(weights_theta_pi[i, j]))


theta: 0.1   pi: 0.5
average return:  441545.7943917054
theta: 0.1   pi: 2.0
average return:  -4827651.62932147
theta: 0.1   pi: 4.0
average return:  129605.26522738372
theta: 0.1   pi: 6.0
average return:  236705.26300256213
theta: 10.0   pi: 0.5
average return:  211361.4633983196
theta: 10.0   pi: 2.0
average return:  -1201793.7304608535
theta: 10.0   pi: 4.0
average return:  565380.5371003367
theta: 10.0   pi: 6.0
average return:  100075.54761797763
theta: 1000.0   pi: 0.5
average return:  8212942.4540810725
theta: 1000.0   pi: 2.0
average return:  -798596.3079099151
theta: 1000.0   pi: 4.0
average return:  -333731.89641582104
theta: 1000.0   pi: 6.0
average return:  140668.569283059
theta: 100000.0   pi: 0.5
average return:  -19235420.53585599
theta: 100000.0   pi: 2.0
average return:  -598444.9383189494
theta: 100000.0   pi: 4.0
average return:  -199989.53185309068
theta: 100000.0   pi: 6.0
average return:  9660.447244069897
theta: 1000000.0   pi: 0.5
average return:  -7058170.710